# Working with Jina AI in Pixeltable

Pixeltable's Jina AI integration enables you to access state-of-the-art embedding and reranker models via the Jina AI API.

### Prerequisites
- A Jina AI account with an API key (https://jina.ai/)

### Important notes

- Jina AI usage may incur costs based on your Jina AI plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.


First you'll need to install Pixeltable and set up your Jina AI API key.


In [ ]:
%pip install -qU pixeltable


In [ ]:
import os
import getpass

if 'JINA_API_KEY' not in os.environ:
    os.environ['JINA_API_KEY'] = getpass.getpass('Enter your Jina AI API key: ')


Now let's create a Pixeltable directory to hold the tables for our demo.


In [ ]:
import pixeltable as pxt

# Remove the 'jina_demo' directory and its contents, if it exists
pxt.drop_dir('jina_demo', force=True)
pxt.create_dir('jina_demo')


## Text Embeddings

Jina AI provides frontier multilingual embedding models for semantic search and RAG applications. The `jina-embeddings-v3` model supports 89+ languages and achieves state-of-the-art performance.


In [ ]:
from pixeltable.functions import jina

# Create a table for document embeddings
docs_t = pxt.create_table('jina_demo.documents', {'text': pxt.String})

# Add computed column with Jina embeddings
# task='retrieval.passage' optimizes embeddings for documents to be searched
docs_t.add_computed_column(
    embedding=jina.embeddings(
        docs_t.text,
        model='jina-embeddings-v3',
        task='retrieval.passage'
    )
)


In [ ]:
# Insert some sample documents
documents = [
    "The Mediterranean diet emphasizes fish, olive oil, and vegetables, believed to reduce chronic diseases.",
    "Photosynthesis in plants converts light energy into glucose and produces essential oxygen.",
    "20th-century innovations, from radios to smartphones, centered on electronic advancements.",
    "Rivers provide water, irrigation, and habitat for aquatic species, vital for ecosystems.",
    "Apple's conference call to discuss fourth fiscal quarter results is scheduled for Thursday, November 2, 2023.",
    "Shakespeare's works, like 'Hamlet' and 'A Midsummer Night's Dream,' endure in literature."
]

docs_t.insert({'text': doc} for doc in documents)


In [ ]:
# View the embeddings
docs_t.select(docs_t.text, docs_t.embedding).head(3)


## Multilingual Embeddings

Jina AI models excel at multilingual text. The same model can embed text in different languages into the same semantic space.


In [ ]:
# Create a table for multilingual content
multilingual_t = pxt.create_table('jina_demo.multilingual', {'text': pxt.String, 'language': pxt.String})

multilingual_t.add_computed_column(
    embedding=jina.embeddings(
        multilingual_t.text,
        model='jina-embeddings-v3',
        task='text-matching'
    )
)

# Insert texts in different languages (all about organic skincare)
multilingual_t.insert([
    {'text': 'Organic skincare for sensitive skin with aloe vera and chamomile.', 'language': 'English'},
    {'text': 'Bio-Hautpflege für empfindliche Haut mit Aloe Vera und Kamille.', 'language': 'German'},
    {'text': 'Cuidado de la piel orgánico para piel sensible con aloe vera y manzanilla.', 'language': 'Spanish'},
    {'text': '针对敏感肌专门设计的天然有机护肤产品', 'language': 'Chinese'},
])

multilingual_t.select(multilingual_t.language, multilingual_t.text).collect()


## Embedding Index for Similarity Search

You can use Jina AI embeddings with Pixeltable's embedding index for efficient similarity search.


In [ ]:
# Create a table with an embedding index
search_t = pxt.create_table('jina_demo.search', {'text': pxt.String})

# Add embedding index for similarity search
embed_fn = jina.embeddings.using(model='jina-embeddings-v3', task='retrieval.passage')
search_t.add_embedding_index('text', string_embed=embed_fn)

# Insert documents
search_t.insert({'text': doc} for doc in documents)


In [ ]:
# Perform similarity search
sim = search_t.text.similarity(string="What are the health benefits of Mediterranean food?")
search_t.order_by(sim, asc=False).limit(3).select(search_t.text, score=sim).collect()


## Reranking

Jina AI's reranker models can improve search relevance by reordering results based on semantic similarity to the query.


In [ ]:
# Create a table for reranking queries
rerank_t = pxt.create_table('jina_demo.rerank', {'query': pxt.String, 'documents': pxt.Json})

# Add computed column for reranking
rerank_t.add_computed_column(
    reranked=jina.rerank(
        rerank_t.query,
        rerank_t.documents,
        model='jina-reranker-v2-base-multilingual',
        top_n=3,
        return_documents=True
    )
)

# Insert a query with candidate documents
rerank_t.insert(
    query="When is Apple's conference call scheduled?",
    documents=documents
)


In [ ]:
# View the reranked results
result = rerank_t.select(rerank_t.reranked).collect()
result['reranked'][0]


## Available Models

### Embedding Models

| Model | Context | Dimensions | Description |
|-------|---------|------------|-------------|
| `jina-embeddings-v4` | 32K | 2048 | Universal multimodal multilingual |
| `jina-embeddings-v3` | 8K | 1024 | Frontier multilingual (89+ languages) |
| `jina-clip-v2` | 8K | 1024 | Multimodal (text + images) |
| `jina-embeddings-v2-base-code` | 8K | 768 | Code and docstring search |
| `jina-code-embeddings-1.5b` | 32K | 1536 | Efficient code embeddings |

### Reranker Models

| Model | Description |
|-------|-------------|
| `jina-reranker-v2-base-multilingual` | Cross-encoder with multilingual support |
| `jina-reranker-v1-turbo-en` | Fast and accurate English reranker |
| `jina-reranker-v1-tiny-en` | Fastest English reranker |
| `jina-colbert-v2` | ColBERT-based multilingual reranker |

### Task Types for Embeddings

The `task` parameter optimizes embeddings for specific use cases:

- `retrieval.query`: For search queries
- `retrieval.passage`: For documents/passages to be searched
- `text-matching`: For semantic similarity
- `classification`: For classification tasks
- `separation`: For clustering/separation tasks


## Learn More

- [Jina AI Documentation](https://jina.ai/)
- [Jina Embeddings](https://jina.ai/embeddings/)
- [Jina Reranker](https://jina.ai/reranker/)
- [API Rate Limits](https://jina.ai/api-dashboard/rate-limit)
